# Model Training
Welcome to the 'Model Training and Prediction' notebook, a crucial facet of our project's data science pipeline. In this notebook, we offer a meticulous examination of our rigorous model development process. The pipeline starts by accepting training data, followed by fitting three distinct types of models to it: Random Forest, Gradient Boosted Tree, and XGBoost. The initial stages include encoding categorical variables and executing Recursive Feature Elimination (RFE) for feature selection. This is succeeded by the application of genetic algorithms to hyperparameter tuning, operating in tandem with a cross-validation routine. Subsequently, the best model is selected based on the highest F1 score, indicating the balance between precision and recall. Finally, the selected model is utilized to predict the outcomes for the current week's round of NRL matches. This process is iterative and cyclical, with the potential for revisiting earlier stages based on the model's performance. Let us proceed with this in-depth exploration.

## Set up Environment
he initial section of the code is dedicated to preparing the environment for our model training pipeline. Key Python libraries, including sys, pandas, sqlite3, pathlib, and numpy, are imported to manage system parameters, perform data operations, handle database connectivity, manage file paths, and conduct numerical operations respectively. Additionally, the path of our custom modules, residing in the 'model-training/functions' directory, is appended to the system path, enabling us to import the modelling_functions and training_config modules. These modules contain custom functions and configuration settings essential for the subsequent phases of data preprocessing, model training, and prediction. This setup ensures all necessary tools and functions are readily available for the pipeline's operations.

In [1]:
# %pip install pandas numpy
#!pip3 install -r requirements.txt
#%pip install -r /Users/levonrush/Documents/GitHub/footy-tipper/requirements.txt

In [1]:
import sys
import pandas as pd
import sqlite3
import pathlib
import numpy as np
from joblib import dump

sys.path.append("functions") 
import modelling_functions as mf
import model_properties as mp
import training_config as tc

## Get data
Our process starts by establishing the root directory of the project and constructing the relative path to the 'footy-tipper-db.sqlite' database located within the 'data' directory. We then connect to this SQLite database and use a SQL query housed in the 'footy_tipping_data.sql' file, found in the 'sql' directory, to extract the required data. This data is loaded into a pandas DataFrame, footy_tipping_data, serving as the basis for our subsequent modeling activities. Upon successful extraction of the data, we ensure the database connection is closed, maintaining good coding practice and resource management.

In [2]:
# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"

# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read SQL query from external SQL file
with open('sql/training_data.sql', 'r') as file:
    query = file.read()

training_data = pd.read_sql_query(query, con)

# Don't forget to close the connection
con.close()

training_data

,game_id,round_id,round_name,game_number,game_state_name,start_time,start_time_utc,venue_name,city,crowd,...,matchup_form,state_of_origin,home_elo,away_elo,elo_diff,home_elo_prob,away_elo_prob,elo_draw_prob,elo_prob_diff,home_ground_advantage
0,2.020111e+10,1.0,Round 1,1.0,Final,1.584044e+09,1.584004e+09,CommBank Stadium,Sydney,21363.0,...,1.0,0.0,1510.543428,1496.559524,13.983904,0.513521,0.466892,0.019587,0.046629,9.877229
1,2.020111e+10,1.0,Round 1,2.0,Final,1.584122e+09,1.584083e+09,GIO Stadium,Canberra,10610.0,...,-1.0,0.0,1515.710577,1466.529411,49.181166,0.551101,0.409164,0.039735,0.141938,8.305680
2,2.020111e+10,1.0,Round 1,3.0,Final,1.584126e+09,1.584090e+09,Queensland Country Bank Stadium,Townsville,22459.0,...,1.0,0.0,1490.406034,1483.396877,7.009157,0.503694,0.476719,0.019587,0.026975,3.062016
3,2.020111e+10,1.0,Round 1,4.0,Final,1.584198e+09,1.584158e+09,McDonald Jones Stadium,Newcastle,10239.0,...,1.0,0.0,1484.839412,1486.546449,-1.707037,0.491400,0.489014,0.019587,0.002386,5.768794
4,2.020111e+10,1.0,Round 1,5.0,Final,1.584207e+09,1.584167e+09,Accor Stadium,Sydney,NaN,...,4.0,0.0,1505.190325,1506.316264,-1.125939,0.492220,0.488194,0.019587,0.004026,3.141187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,2.023111e+10,20.0,Round 20,3.0,Final,1.689442e+09,1.689406e+09,4 Pines Park,Sydney,13240.0,...,-3.0,0.0,1490.082965,1541.085361,-51.002395,0.417866,0.552283,0.029851,-0.134417,-2.280337
715,2.023111e+10,20.0,Round 20,4.0,Final,1.689450e+09,1.689414e+09,Sydney Cricket Ground,Sydney,12021.0,...,-3.0,0.0,1480.833645,1526.299892,-45.466247,0.425463,0.544686,0.029851,-0.119223,-3.661337
716,2.023111e+10,20.0,Round 20,5.0,Final,1.689523e+09,1.689480e+09,Go Media Stadium,Auckland,24012.0,...,-1.0,0.0,1528.936608,1528.030248,0.906359,0.495087,0.485326,0.019587,0.009761,1.569303
717,2.023111e+10,20.0,Round 20,6.0,Final,1.689524e+09,1.689488e+09,Kayo Stadium,Redcliffe,NaN,...,0.0,0.0,1474.548764,1548.389112,-73.840348,0.391889,0.590720,0.017391,-0.198832,-12.737903


## Modelling
During the modeling phase, we invoke the train_and_select_best_model function from our modelling_functions module. This function, by accepting our footy tipping data, predictor variables, outcome variable, and several configuration settings (including Recursive Feature Elimination (RFE) usage, cross-validation folds number, and optimization metric) from the training_config module, initiates the training of three distinct models: Random Forest, Gradient Boosted Tree, and XGBoost. Each model undergoes hyperparameter tuning via genetic algorithms and is evaluated through cross-validation. The best_model, defined by the superior performance on the chosen optimization metric, is selected and, together with X_inference (transformed feature matrix), label_encoder (for encoding categorical variables), and game_id_inference (for associating predictions with specific games), is returned, ready for the prediction phase.

In [14]:
best_model, label_encoder = mf.train_and_select_best_model(
    training_data, tc.predictors, tc.outcome_var,
    tc.use_rfe, tc.num_folds, tc.opt_metric
)

footy_tipper = best_model

footy_tipper

Training model: XGBClassifier
Number of features selected: 51 out of 168
gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	5     	0.683162	0.0119191  	0.692677   	0.660625   
1  	10    	0.687634	0.0113997  	0.699621   	0.671824   
2  	10    	0.694042	0.00475609 	0.699621   	0.688442   
3  	10    	0.691807	0.00729829 	0.699621   	0.680109   
{'n_estimators': 185, 'learning_rate': 0.4953659353767557, 'max_depth': 4, 'subsample': 0.8129602367393964, 'colsample_bytree': 0.48995432746196876, 'gamma': 0.0701691597873137}
0.6996212121212121


GASearchCV(crossover_probability=0.7,
           cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
           estimator=XGBClassifier(base_score=None, booster=None,
                                   callbacks=None, colsample_bylevel=None,
                                   colsample_bynode=None,
                                   colsample_bytree=0.48995432746196876,
                                   early_stopping_rounds=None,
                                   enable_categorical=False, eval_metric=None,
                                   feature_types=None, gamma=0.07016...
                       'learning_rate': <sklearn_genetic.space.space.Continuous object at 0x286ca4d10>,
                       'max_depth': <sklearn_genetic.space.space.Integer object at 0x286ca4b10>,
                       'n_estimators': <sklearn_genetic.space.space.Integer object at 0x286ca6310>,
                       'subsample': <sklearn_genetic.space.space.Continuous object at 0x286ca5ad0>},
           population_size=5, return_train_score=True, scoring='accuracy',
           tournament_size=2)

In [15]:
# feature_importance_df = mp.get_sorted_feature_importance(best_model, X_inference.columns)
# feature_importance_df

## Save Model

In [17]:
dump(label_encoder, project_root / "models" / 'label_encoder.pkl')
dump(footy_tipper, project_root / "models" / 'footy_tipper.pkl')

TypeError: cannot pickle 'module' object